In [37]:

import numpy as np
import matplotlib.pyplot as plt
import gstools as gs
from sklearn.preprocessing import KBinsDiscretizer

import pandas as pd

# 生成数据
x = np.linspace(0, 25, 25)
y = np.linspace(0, 25, 25)
XX, YY = np.meshgrid(x, y)

# 第一个数据
model = gs.Gaussian(dim=2, var=1, len_scale=3)
srf = gs.SRF(model, mean=0.)
srf((x, y), mesh_type='structured')
B0 = srf((x, y), mesh_type='structured', seed=100)



# 定义状态数
n_states = 5
est = KBinsDiscretizer(n_bins=n_states, encode='ordinal', strategy='uniform')
B_discretized = est.fit_transform(B0)

def process_data_with_randomness(B_discretized):
    # 构建转移概率矩阵
    transition_matrix = np.zeros((n_states, n_states))

    for month in range(24):  # 24次状态转移
        for day in range(25):
            current_state = int(B_discretized[month, day])
            next_state = int(B_discretized[month + 1, day])
            transition_matrix[current_state, next_state] += 1

    # 归一化转移概率矩阵
    transition_matrix /= transition_matrix.sum(axis=1, keepdims=True)

    # 使用马尔可夫链预测第26个月的状态
    last_month_states = B_discretized[-1, :]
    predicted_states = []

    for state in last_month_states:
        predicted_state = np.random.choice(np.arange(n_states), p=transition_matrix[int(state)])
        predicted_states.append(predicted_state)

    print(predicted_states)
    return predicted_states
    # 预测值从状态返回到实际值范围
    # B26_pred = est.inverse_transform([predicted_states])
    # return B26_pred.flatten()

all_processed_values=[]

for i in range(1, 2):
    processed_values = process_data_with_randomness(B_discretized)
    all_processed_values.append(pd.Series(processed_values, name=f'Out0'))

results_df_initial = pd.DataFrame()
# 将所有列一次性连接到 results_df 中
results_df_initial = pd.concat(all_processed_values, axis=1)
results_df_initial.to_csv("initial-out/pred_nonoisy_initial.csv", index=False)
# 检查结果

all_processed_values=[]
# 运行过程 1000 次并存储结果
for i in range(1, 1001):
    print(i)
    processed_values = process_data_with_randomness(B_discretized)
    all_processed_values.append(pd.Series(processed_values, name=f'Out{i}'))

# Initialize the dataframe to store results
results_df = pd.DataFrame()
# 将所有列一次性连接到 results_df 中
results_df = pd.concat(all_processed_values, axis=1)
results_df.to_csv("repro-out/pred_nonoisy.csv", index=False)
# 检查结果
print(results_df)

    Out0    value0
0      1  0.799792
1      1  0.923805
2      4  0.366569
3      4  0.493110
4      0  0.019364
5      4  0.515354
6      2 -0.703104
7      2 -1.349441
8      2 -0.561570
9      0 -0.281873
10     2 -0.667582
11     3  1.412527
12     1  1.006121
13     0 -0.315196
14     2 -0.501593
15     0 -0.190967
16     1  0.790172
17     4  0.175016
18     0 -0.470413
19     0 -0.065426
20     0 -0.025528
21     4  0.429970
22     0 -0.451267
23     3  1.228292
24     4  0.168110
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
15